In [2]:

from dask_jobqueue import SLURMCluster # pip 
from dask.distributed import Client, progress 
import dask


import matplotlib as mpl
# Define Agg as Backend for matplotlib when no X server is running
mpl.use('Agg')
import socket
import os
import sys
import importlib    

In [3]:

def squeue_user(username = "$USER"):
    _squeue_user = os.system("squeue --user="+str(username))
    return _squeue_user 

with os.popen("pwd ") as f:
    _pwd = f.readline()

In [4]:
cluster = SLURMCluster(
    name='dask-cluster', 
    cores=256,    
    memory="500 GB", 
    project="bb1153",
    queue= "compute", 
    #interface='ib0',
    walltime='04:30:50',
)
client = Client(cluster)
print(cluster.job_script())

/home/b/b382267/mambaforge/envs/aqua/lib/python3.10/site-packages/dask_jobqueue/slurm.py:49: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)


#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p compute
#SBATCH -A bb1153
#SBATCH -n 1
#SBATCH --cpus-per-task=256
#SBATCH --mem=466G
#SBATCH -t 04:30:50

/home/b/b382267/mambaforge/envs/aqua/bin/python -m distributed.cli.dask_worker tcp://136.172.124.7:39953 --nthreads 16 --nworkers 16 --memory-limit 29.10GiB --name dummy-name --nanny --death-timeout 60



/home/b/b382267/mambaforge/envs/aqua/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35893 instead
  warnings.warn(
/home/b/b382267/mambaforge/envs/aqua/lib/python3.10/site-packages/dask_jobqueue/slurm.py:49: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)


In [5]:
cluster.scale(jobs=1)



In [7]:
squeue_user()

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           4257997   compute dask-wor  b382267  R       1:23      1 l40125


0

In [8]:
os.system("scancel   4250481") 

0

In [16]:
import diagnostics.tropical_rainfall.tr_pr_mod
from diagnostics.tropical_rainfall import tr_pr_mod

In [58]:
try:
    importlib.reload(tr_pr_mod)
except NameError:
    None

from  diagnostics.tropical_rainfall.tr_pr_mod import TR_PR_Diagnostic
diag = tr_pr_mod.TR_PR_Diagnostic()

diag.num_of_bins = 15
diag.first_edge = 0
diag.width_of_bin = 2000*10**(-6)/diag.num_of_bins

last_edge = diag.first_edge  + diag.num_of_bins*diag.width_of_bin

In [18]:
from aqua import Reader
from aqua.reader import catalogue

In [19]:
import xarray as xr 
import matplotlib.pyplot as plt
import numpy as np 

In [20]:
catalogue()

IFS	tco3999-ng5	2.8km experiment, coupled with FESOM
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- interpolated_global	
	- interpolated_np	
	- interpolated_sp	
	- interpolated_sp_ci	
IFS	tco2559-ng5	4km experiment, coupled with FESOM
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- interpolated_global	
	- interpolated_np	
	- interpolated_sp	
IFS	tco1279-orca025	9km baseline, coupled to NEMO, deep conv ON
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	

FESOM	tco3999-ng5	2.5km experiment, coupuled with IFS
	- elem_grid	
	- node_grid	
	- np	nearest-neighbor interpolation to lat-lon grid
	- interpolated_global2d	
	- interpolated_global_TS	
	- interpolated_global_UV	
	- interpolated_np	
	- interpolated_sp	
	- original_2d	original 2d output
	- original_3d	original 3d output
FESOM	tco2559-ng5	5km experiment, coupuled with IFS
	- elem_grid	
	- node_grid	
	- interpolated_global2d	
	- interpolated_global_TS	
	- interpolated_global_UV	
	- interpolated_np	
	- interpolated_sp	
	- original_2d	orig

config:
  args:
    path: ./config/catalog.yaml
  description: ''
  driver: intake.catalog.local.YAMLFileCatalog
  metadata: {}


# ICON data

In [66]:
reader = Reader(model="ICON", exp="ngc2009", source="atm_2d_ml_R02B06")
#data = reader.retrieve(fix=False)
#data = reader.retrieve(apply_unit_fix=False)
ICON_2009 = reader.retrieve()

No fixes defined for model  ICON


In [67]:
ICON_2009_small = ICON_2009['pr'][0:100,:]
ICON_2009_small = ICON_2009_small.compute()

In [68]:
diag.time_interpreter_b(ICON_2009)

diag.time_interpreter_b(ICON_2009_small) 

timestep is 30 minutes
timestep is 30 minutes


'MINUTE'

In [ ]:
ICON_2009_small

In [108]:
diag.coordinate_names(ICON_2009_small)

(None, None)

### Fastest histogram

In [165]:
hist_fast_ICON,  size_ICON,   del_time_fast_ICON  = diag.hist1d_fast(ICON_2009_small,  preprocess = False)
hist_fast_ICON

<xarray.DataArray (bin: 15)>
array([3.1195073e+07, 6.4687600e+05, 2.8013100e+05, 1.6057400e+05,
       1.0343300e+05, 7.2720000e+04, 5.3261000e+04, 4.0553000e+04,
       3.1905000e+04, 2.5767000e+04, 2.0959000e+04, 1.7618000e+04,
       1.4783000e+04, 1.2788000e+04, 1.0624000e+04])
Coordinates:
  * bin      (bin) float64 0.0 0.0001333 0.0002667 ... 0.0016 0.001733 0.001867

In [166]:

fig = plt.figure()
ax = fig.add_subplot(111)

_pdf = True

diag.hist_plot(hist_fast_ICON,           pdf=_pdf,   color = 'tab:red',     label='fast')

ax.grid()

plt.savefig("./figures/pdf_hists_ICON.png",
            bbox_inches ="tight",
            pad_inches = 1,
            transparent = True,
            facecolor ="w",
            edgecolor ='w',
            orientation ='landscape')

hist_plot function in the process 



In [69]:
fig = plt.figure()
plt.plot(diag.mean_per_timestep(ICON_2009_small), label= 'mean')
plt.plot(diag.median_per_timestep(ICON_2009_small), label='median')

plt.yscale('log')
plt.xlabel('Time', fontsize=12)
plt.ylabel('Precipitation per day', fontsize=12)
plt.title('Mean/median values of precipitation', fontsize =16)
plt.legend(fontsize=12)

#plt.yscale('log')
print('gmean ....')


plt.savefig("./figures/mean_and_median_ICON.png",
            bbox_inches ="tight",
            pad_inches = 1,
            transparent = True,
            facecolor ="w",
            edgecolor ='w',
            orientation ='landscape')

gmean ....


#  MSWEP 

In [61]:
reader = Reader(model="MSWEP", exp="past", source="monthly")
#data = reader.retrieve(fix=False)
#data = reader.retrieve(apply_unit_fix=False)
MSWEP = reader.retrieve()

precipitation: corrected multiplying by density of water 1000 kg m-3
precipitation: incommensurate units converting mm month-1 to kg m-2 s-1 --> dimensionless


In [62]:
MSWEP 

MSWEP_small = MSWEP['pr'][0:20,:]
MSWEP_small = MSWEP_small.compute()

In [63]:
diag.time_interpreter_b(MSWEP)

diag.time_interpreter_b(MSWEP_small) 

timestep is month
timestep is month


'M'

In [148]:
hist_fast_MSWEP,  size_MSWEP,   del_time_fast_MSWEP  = diag.hist1d_fast(MSWEP_small)
hist_fast_MSWEP

<xarray.DataArray (bin: 15)>
array([1.3122303e+07, 1.2379830e+06, 3.5975000e+04, 3.1350000e+03,
       4.5500000e+02, 8.1000000e+01, 2.6000000e+01, 2.8000000e+01,
       6.0000000e+00, 1.0000000e+00, 4.0000000e+00, 2.0000000e+00,
       1.0000000e+00, 0.0000000e+00, 0.0000000e+00])
Coordinates:
  * bin      (bin) float64 0.0 0.0001333 0.0002667 ... 0.0016 0.001733 0.001867

In [152]:

fig = plt.figure()
ax = fig.add_subplot(111)

_pdf = True

diag.hist_plot(hist_fast_MSWEP,           pdf=_pdf,   color = 'tab:red',     label='fast')

ax.grid()

plt.savefig("./figures/pdf_hists_MSWEP.png",
            bbox_inches ="tight",
            pad_inches = 1,
            transparent = True,
            facecolor ="w",
            edgecolor ='w',
            orientation ='landscape')

hist_plot function in the process 



In [64]:
fig = plt.figure()
plt.plot(diag.mean_per_timestep(MSWEP_small), label= 'mean')
plt.plot(diag.median_per_timestep(MSWEP_small), label='median')

plt.yscale('log')
plt.xlabel('Time', fontsize=12)
plt.ylabel('Precipitation per day', fontsize=12)
plt.title('Mean/median values of precipitation', fontsize =16)
plt.legend(fontsize=12)

#plt.yscale('log')
print('gmean ....')


plt.savefig("./figures/mean_and_median_MSWEP.png",
            bbox_inches ="tight",
            pad_inches = 1,
            transparent = True,
            facecolor ="w",
            edgecolor ='w',
            orientation ='landscape')

gmean ....


### Animation

In [ ]:
diag.animated(MSWEP,  time_ind_max = 10, label = 'MSWEP')

In [173]:

scale_factor = 90/diag.trop_lat
fig = plt.figure(figsize=(8*scale_factor,6))
diag.ds_per_lat_range(MSWEP)['pr'][0,:,:].plot()#vmin=0, vmax=100)


plt.savefig("./figures/trop_rains_MSWEP.png",
            bbox_inches ="tight",
            pad_inches = 1,
            transparent = True,
            facecolor ="w",
            edgecolor ='w',
            orientation ='landscape')

# IFS

In [21]:
reader = Reader(model="IFS", exp="tco3999-ng5", source="ICMGG_atm2d")
#data = reader.retrieve(fix=False)
#data = reader.retrieve(apply_unit_fix=False)
IFS = reader.retrieve()

tp: corrected multiplying by density of water 1000 kg m-3
tp: corrected dividing by accumulation time 10800 s


In [22]:
IFS 

<xarray.Dataset>
Dimensions:  (time: 5401, value: 64144000, level: 5)
Coordinates:
    lat      (value) float64 dask.array<chunksize=(64144000,), meta=np.ndarray>
  * level    (level) int64 1 2 3 4 5
    lon      (value) float64 dask.array<chunksize=(64144000,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-01-20 2020-01-20T01:00:00 ... 2020-09-01
Dimensions without coordinates: value
Data variables: (12/81)
    100si    (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    100u     (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    100v     (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    10u      (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    10v      (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    2d       (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    ...       ...
    uvb      (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    vike     (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    vipie    (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    vipile   (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    vithe    (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
    z        (time, value) float64 dask.array<chunksize=(1, 64144000), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

In [23]:
IFS_small = IFS['pr'][0:10,:]
IFS_small = IFS_small.compute()

In [24]:
diag.time_interpreter_b(IFS)

diag.time_interpreter_b(IFS_small) 

timestep is 1 hours
timestep is 1 hours


'H'

In [25]:
hist_fast_IFS,  size_IFS,   del_time_fast_IFS  = diag.hist1d_fast(IFS_small, preprocess = False)
hist_fast_IFS

<xarray.DataArray (bin: 15)>
array([6.07995876e+08, 1.28851960e+07, 5.78830800e+06, 3.38266200e+06,
       2.31160500e+06, 1.66320900e+06, 1.25000600e+06, 9.51033000e+05,
       7.23704000e+05, 5.73352000e+05, 4.63395000e+05, 3.88526000e+05,
       3.30936000e+05, 2.80356000e+05, 2.41592000e+05])
Coordinates:
  * bin      (bin) float64 0.0 0.0001333 0.0002667 ... 0.0016 0.001733 0.001867

In [48]:

fig = plt.figure()
ax = fig.add_subplot(111)

_pdf = True

diag.hist_plot(hist_fast_IFS,           pdf=_pdf,   color = 'tab:red',     label='fast')

ax.grid()

plt.savefig("./figures/pdf_hists_IFS.png",
            bbox_inches ="tight",
            pad_inches = 1,
            transparent = True,
            facecolor ="w",
            edgecolor ='w',
            orientation ='landscape')

hist_plot function in the process 



In [59]:
fig = plt.figure()
plt.plot(diag.mean_per_timestep(IFS_small), label= 'mean')
plt.plot(diag.median_per_timestep(IFS_small), label='median')

plt.yscale('log')
plt.xlabel('Time', fontsize=12)
plt.ylabel('Precipitation per day', fontsize=12)
plt.title('Mean/median values of precipitation', fontsize =16)
plt.legend(fontsize=12)

#plt.yscale('log')
print('gmean ....')


plt.savefig("./figures/mean_and_median_IFS.png",
            bbox_inches ="tight",
            pad_inches = 1,
            transparent = True,
            facecolor ="w",
            edgecolor ='w',
            orientation ='landscape')

gmean ....


# FESOM

In [213]:

reader = Reader(model="FESOM", exp="tco3999-ng5", source="interpolated_np")
#data = reader.retrieve(fix=False)
#data = reader.retrieve(apply_unit_fix=False)
FESOM = reader.retrieve()

Source areas file not found: /work/bb1153/b382075/nextgems/weights/areas_FESOM_tco3999-ng5_interpolated_np.nc
Attempting to generate it ...
Source grid:  /work/bb1153/b382075/nextgems/grids/fesom_tco2559_interpolated_2d.nc
Success!


In [215]:
print(FESOM)

<xarray.Dataset>
Dimensions:    (depth: 1, lat: 2000, time: 5400, lon: 2000)
Coordinates:
  * depth      (depth) float64 0.0
  * lat        (lat) float64 -3.332e+06 -3.329e+06 ... 3.423e+06 3.426e+06
    latitude   (time, lat, lon) float64 dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
  * lon        (lon) float64 -3.403e+06 -3.399e+06 ... 3.394e+06 3.397e+06
    longitude  (time, lat, lon) float64 dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
  * time       (time) datetime64[ns] 2020-01-20T00:56:00 ... 2020-08-31T23:56:00
Data variables:
    m_ice      (time, depth, lat, lon) float32 dask.array<chunksize=(1, 1, 2000, 2000), meta=np.ndarray>
Attributes: (12/21)
    FESOM_ClimateDataPath:               /ec/res4/scratch/natr/RAPS/20/inputs...
    FESOM_MeshPath:                      /ec/res4/scratch/natr/RAPS/20/inputs...
    FESOM_evp_rheol_steps:               120
    FESOM_force_rotation:                -1
    FESOM_git_SHA:                       469e74b1
    FESOM_in

In [199]:
FESOM

FESOM_small = FESOM['2t'][0:20,:]
#FESOM_small = FESOM_small.compute()

In [202]:
diag.time_interpreter_b(FESOM)

diag.time_interpreter_b(FESOM_small) 

timestep is 1 hours
timestep is 1 hours


'H'

In [209]:
FESOM_small.stack(temp_c = ['time', 'lat', 'lon'])

<xarray.DataArray '2t' (temp_c: 80000000)>
dask.array<reshape, shape=(80000000,), dtype=float32, chunksize=(4000000,), chunktype=numpy.ndarray>
Coordinates:
    latitude   (temp_c) float64 dask.array<chunksize=(4000000,), meta=np.ndarray>
    longitude  (temp_c) float64 dask.array<chunksize=(4000000,), meta=np.ndarray>
  * temp_c     (temp_c) object MultiIndex
  * time       (temp_c) datetime64[ns] 2020-01-20 ... 2020-01-20T19:00:00
  * lat        (temp_c) float64 9.077e+22 9.077e+22 ... 1.148e+24 1.148e+24
  * lon        (temp_c) float64 7.678e-193 9.607e-251 ... 5.179e+22 6.748e+22

In [208]:
temp = 0
temp_c = 'time'
for i in FESOM_small.coords:
    print(i)
    temp = FESOM_small.stack(temp_c = [temp_c, i])
    

lat
latitude


KeyError: 'latitude'

In [204]:
FESOM_small

<xarray.DataArray '2t' (time: 20, lat: 2000, lon: 2000)>
dask.array<getitem, shape=(20, 2000, 2000), dtype=float32, chunksize=(1, 2000, 2000), chunktype=numpy.ndarray>
Coordinates:
  * lat        (lat) float64 9.077e+22 1.221e+23 ... 8.986e+23 1.148e+24
    latitude   (time, lat, lon) float64 dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
  * lon        (lon) float64 7.678e-193 9.607e-251 ... 5.179e+22 6.748e+22
    longitude  (time, lat, lon) float64 dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
  * time       (time) datetime64[ns] 2020-01-20 ... 2020-01-20T19:00:00

In [210]:
hist_fast_FESOM,  size_FESOM,   del_time_fast_FESOM  = diag.hist1d_fast(FESOM_small)
hist_fast_FESOM 

ValueError: When changing to a larger dtype, its size must be a divisor of the total size in bytes of the last axis of the array.

# CERES

In [189]:
reader = Reader(model="CERES", exp="syn-toa41", source="daily")
#data = reader.retrieve(fix=False)
#data = reader.retrieve(apply_unit_fix=False)
CERES = reader.retrieve()

No fixes defined for model  CERES


In [190]:
CERES


<xarray.Dataset>
Dimensions:            (time: 7670, lon: 360, lat: 180)
Coordinates:
  * time               (time) datetime64[ns] 2001-01-01T12:00:00 ... 2021-12-...
  * lon                (lon) float32 0.5 1.5 2.5 3.5 ... 356.5 357.5 358.5 359.5
  * lat                (lat) float32 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
Data variables:
    toa_sw_all_daily   (time, lat, lon) float32 dask.array<chunksize=(2443, 180, 360), meta=np.ndarray>
    toa_lw_all_daily   (time, lat, lon) float32 dask.array<chunksize=(2443, 180, 360), meta=np.ndarray>
    toa_net_all_daily  (time, lat, lon) float32 dask.array<chunksize=(2443, 180, 360), meta=np.ndarray>
Attributes:
    title:        CERES SYN1deg Daily data
    institution:  NASA Langley Research Center
    Conventions:  CF-1.4
    comment:       
    Version:      Edition 4.1: Release Date August 22, 2019
    DOI:          10.5067/Terra+Aqua/CERES/SYN1degDay_L3.004A
    Fill_Value:   Fill Value is -999.0

In [187]:
CERES_small = CERES['cldpress_total_daynight_mon'][0:20,:]
CERES_small = CERES_small.compute()

In [188]:
CERES_small

<xarray.DataArray 'cldpress_total_daynight_mon' (time: 20, lat: 180, lon: 360)>
array([[[540.8774 , 540.8774 , 540.8774 , ..., 540.8774 , 540.8774 ,
         540.8774 ],
        [512.0556 , 512.0556 , 512.0556 , ..., 512.0556 , 512.0556 ,
         512.0556 ],
        [501.4694 , 501.4694 , 501.4694 , ..., 501.4694 , 501.4694 ,
         501.4694 ],
        ...,
        [747.74554, 747.74554, 747.74554, ..., 747.74554, 747.74554,
         747.74554],
        [758.81665, 758.81665, 758.81665, ..., 758.81665, 758.81665,
         758.81665],
        [824.22266, 824.22266, 824.22266, ..., 824.22266, 824.22266,
         824.22266]],

       [[581.8431 , 581.8431 , 581.8431 , ..., 581.8431 , 581.8431 ,
         581.8431 ],
        [554.3841 , 554.3841 , 554.3841 , ..., 554.3841 , 554.3841 ,
         554.3841 ],
        [576.3648 , 576.3648 , 576.3648 , ..., 576.3648 , 576.3648 ,
         576.3648 ],
...
        [685.4701 , 685.4701 , 685.4701 , ..., 685.4701 , 685.4701 ,
         685.4701 ],
        [699.5922 , 699.5922 , 699.5922 , ..., 699.5922 , 699.5922 ,
         699.5922 ],
        [745.5259 , 745.5259 , 745.5259 , ..., 745.5259 , 745.5259 ,
         745.5259 ]],

       [[408.49155, 408.49155, 408.49155, ..., 408.49155, 408.49155,
         408.49155],
        [411.685  , 411.685  , 411.685  , ..., 411.685  , 411.685  ,
         411.685  ],
        [416.28476, 416.28476, 416.28476, ..., 416.28476, 416.28476,
         416.28476],
        ...,
        [725.33685, 725.33685, 725.33685, ..., 725.33685, 725.33685,
         725.33685],
        [723.205  , 723.205  , 723.205  , ..., 723.205  , 723.205  ,
         723.205  ],
        [795.37994, 795.37994, 795.37994, ..., 795.37994, 795.37994,
         795.37994]]], dtype=float32)
Coordinates:
  * lon      (lon) float32 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
  * lat      (lat) float32 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * time     (time) datetime64[ns] 2000-03-15 2000-04-15 ... 2001-10-15
Attributes:
    long_name:      Cloud Effective Pressure, Daytime-and-Nighttime condition...
    standard_name:  Cloud Effective Pressure - Daytime-and-Nighttime
    CF_name:        none
    comment:        none
    units:          hPa
    valid_min:            0.00000
    valid_max:            1050.00